In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


llm = ChatOpenAI(
    temperature=0.3,
    # model="gpt-4",
    )


def login_tool(login_url: str, id: str, pwd: str, btn_name: str):
    chrome_options =  webdriver.ChromeOptions()
    chrome_options.add_experimental_option("detach", True)
    
    global driver

    driver = webdriver.Chrome(options=chrome_options)
    driver.implicitly_wait(3)

    # 로그인 페이지로 이동
    driver.get(login_url)

    # 로그인 폼 요소 찾기
    id_input = driver.find_element(By.XPATH,'//*[@id="id"]')
    pwd_input = driver.find_element(By.XPATH, '//*[@id="password"]')

    # 사용자 이름과 비밀번호 입력
    id_input.send_keys(id)
    pwd_input.send_keys(pwd)

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{btn_name}')]"))
    )
    element.click()
    return driver


# 마타와 XX 선택할 때
def first_tool(btn_name: str):
    if "마타와" in btn_name:
        cond = f"//a[.//h3[contains(., '{btn_name}')]]"
    elif "[마타와" in btn_name:
        cond = f"//a[contains(., '{btn_name}')]"
    else:
        cond = f"//button[contains(., '{btn_name}')]"
        
    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, cond))
    )
    element.click()
    return driver


# STEP 01. 학년 선택하고 다음으로 이동
def second_tool(click_btn, move_btn):
    element = driver.find_elements(By.XPATH, f"//p[contains(., '{click_btn}')]")
    element[0].click()

    element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    element.click()
    return driver


def third_tool(select_box_btn, grade_btn, selected_class_list, move_btn):
    placeholder = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[contains(text(), '{select_box_btn}')]"))
    )
    placeholder.click()

    grade_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//div[text()='{grade_btn}']"))
    )
    grade_element.click()

    # TODO: 복수 반 선택 로직
    for sc in selected_class_list:
        class_element = driver.find_element(By.XPATH, f"//div[contains(text(), '{sc}')]")
        class_element.click()
    
    move_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{move_btn}')]"))
    )
    move_element.click()
    return driver


agent = initialize_agent(
    llm=llm, 
    verbose=True,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        StructuredTool.from_function(
            func=login_tool,
            name="login tool",
            description="""
            With this tool, you can access a url where login is possible, enter the id and password(pwd), and by pressing a button, you can log in to the page.
            """,
        ), 
        StructuredTool.from_function(
            func=first_tool,
            name="first tool",
            description="""
            This tool is used simply for the action of clicking a single button.
            """,
        ),
        StructuredTool.from_function(
            func=second_tool,
            name="second tool",
            description="""
            This tool is used to select one condition and then move to the next page through a '다음' or '완료' button. A total of two clicks are involved.
            Use this tool in the final step of submission.
            """,
        ),
        StructuredTool.from_function(
            func=third_tool,
            name="third tool",
            description="""
            This tool allows you to first select a target through a placeholder, and then choose various conditions. Therefore, more than three clicks will occur.
            """,
        ),
    ],   
)


prompt = """ 

""" 
agent.invoke(prompt)



> Entering new AgentExecutor chain...
It seems that the user did not provide any input or question. I will wait for the user to provide more information or ask a question.

> Finished chain.


{'input': ' \n\n',
 'output': 'It seems that the user did not provide any input or question. I will wait for the user to provide more information or ask a question.'}